In [1]:
from pathlib import Path

import os
import sys

current_path = Path(os.getcwd())
sys.path.append(str(current_path.parent))

In [2]:
import vectorian

In [3]:
import vectorian
from vectorian.embeddings import CompressedFastTextVectors

fasttext = CompressedFastTextVectors("models/fasttext-en-mini.kv")

import vectorian.utils as utils

token_mappings = {
    "tokenizer": [],
    "tagger": []
}

#token_mappings["tokenizer"].append(utils.lowercase())
token_mappings["tokenizer"].append(utils.erase("W"))
token_mappings["tokenizer"].append(utils.alpha())

def filter_punct(t):
    return None if t["pos"] == "PUNCT" else t

token_mappings["tagger"].append(filter_punct)
token_mappings["tagger"].append(utils.rewrite({
    'pos': {
        'PROPN': 'NOUN'
    },
    'tag': {
        'NNP': 'NN',
        'NNPS': 'NNS',
    }
}))

from vectorian.importers import NovelImporter
from vectorian.session import LabSession
import spacy

nlp = spacy.load("en_core_web_sm")

im = NovelImporter(nlp)
doc = im("/Users/arbeit/Wise Children.txt")

session = LabSession(
    [doc],
    static_embeddings=[fasttext],
    token_mappings=token_mappings,
    embedding_sampling='nearest')

Matching checksum for /Users/arbeit/Projects/vectorian-2021/vectorian/core/cpp/core.cpp --> not compiling


Importing /Users/arbeit/Wise Children.txt: 100%|██████████| 369/369 [00:04<00:00, 87.44it/s] 
Opening fasttext-en-mini: 100%|██████████
Loading fasttext-en-mini: 100%|█████████▉


In [4]:
list(session.embeddings)[0].name

'fasttext-en-mini'

In [5]:
import ipywidgets as widgets
import matplotlib.pyplot as plt

import vectorian.alignment
import vectorian.metrics
import vectorian.session

class FineTuneableWidget:
    def __init__(self, iquery, fix_to=None):
        self._type = widgets.Dropdown(
            options=[x[0] for x in self._types],
            value=self._default if fix_to is None else fix_to,
            description=self._description,
            disabled=fix_to is not None)

        self._iquery = iquery
        
        self._instantiate_fine_tune(self._type.value)
        
        self._type.observe(self.on_changed, names='value')

        box_type = getattr(self, '_box', widgets.VBox)
        self._box = box_type([self._type, self._fine_tune.widget])
        
    def _instantiate_fine_tune(self, name):
        i = [x[0] for x in self._types].index(name)
        self._fine_tune = self._types[i][1](self._iquery)
        
    def on_changed(self, change):
        self._instantiate_fine_tune(change.new)
        self._box.children = [self._type, self._fine_tune.widget]

    def make(self):
        return self._fine_tune.make()
        
    @property
    def widget(self):
        return self._box
    
class CosineMetricWidget:
    def __init__(self, iquery):
        self._vbox = widgets.VBox([])

    @property
    def widget(self):
        return self._vbox
    
    def make(self):
        return vectorian.metrics.CosineMetric()

class ImprovedSqrtCosineMetricWidget:
    def __init__(self, iquery):
        self._vbox = widgets.VBox([])

    @property
    def widget(self):
        return self._vbox

class PNormWidget:
    def __init__(self, iquery):
        self._p = widgets.BoundedFloatText(
            value=2,
            min=1e-4,
            max=10,
            step=0.25,
            description='p:',
            disabled=False)

        self._scale = widgets.BoundedFloatText(
            value=2,
            min=1e-4,
            max=1000,
            step=0.25,
            description='Scale:',
            disabled=False)
        
        self._vbox = widgets.VBox([self._p, self._scale])

    @property
    def widget(self):
        return self._vbox    

    
class VectorMetricWidget(FineTuneableWidget):
    _description = ''
    
    _types = [
        ('Cosine', CosineMetricWidget),
        ('P-Norm', PNormWidget),
        ('Improved Sqrt Cosine', ImprovedSqrtCosineMetricWidget)
    ]
    
    _default = 'Cosine'
    
    def make(self):
        return self._fine_tune.make()
    
    
class EmbeddingWidget:
    def __init__(self, iquery):
        self._iquery = iquery
        
        options = []
        for x in iquery.session.embeddings:
            options.append(x.name)
        if len(options) > 1:
            options.append("mixed")
        
        self._widget = widgets.Dropdown(
            options=options,
            value=options[0],
            description='',
            disabled=False,
        )

    @property
    def widget(self):
        return self._widget

    def make(self):
        for x in self._iquery.session.embeddings:
            if x.name == self._widget.value:
                return x
    

class TokenSimilarityMetricWidget:
    def __init__(self, iquery):
        self._metric = VectorMetricWidget({})
        
        self._embedding = EmbeddingWidget(iquery)
   
        self._hbox = widgets.HBox([
            widgets.Label('Token Metric:'),
            self._metric.widget,
            widgets.Label('on'),
            self._embedding.widget])

    @property
    def widget(self):
        return self._hbox
    
    def make(self):
        return vectorian.metrics.TokenSimilarityMetric(
            self._embedding.make(),
            self._metric.make())


class SlidingGapCostWidget:
    def __init__(self, iquery, description, construct, max=1.0):
        self._construct = construct

        self._cost = widgets.FloatSlider(
            value=0,
            min=0,
            max=max,
            step=0.01,
            description=description,
            disabled=False)
        
        '''
        self._cost = widgets.BoundedFloatText(
            value=0,
            min=0,
            max=max,
            step=0.1,
            description=description,
            disabled=False)
        '''

        self._plot = widgets.Image(
            value=b'',
            format='png',
            width=300,
            height=400,
        )

        self.update_plot()
        self._cost.observe(self.on_changed, names='value')
        
        self._vbox = widgets.VBox([
            self._cost, self._plot], layout=widgets.Layout(border='solid'))
     
    def make(self):
        return self._construct(self._cost.value)
    
    def update_plot(self):
        fig, ax = plt.subplots(1, 1, figsize=(5, 2))
        im_data = self.make().plot_to_image(
            fig, ax, 20, format='png')
        plt.close()
        self._plot.value = im_data

    def on_changed(self, change):
        # cost = change.new
        self.update_plot()
        
    @property
    def widget(self):
        return self._vbox
    
class ConstantGapCostWidget(SlidingGapCostWidget):
    def __init__(self, iquery):
        super().__init__(iquery, 'Cost:', vectorian.alignment.ConstantGapCost)
        
class LinearGapCostWidget(SlidingGapCostWidget):
    def __init__(self, iquery):
        super().__init__(iquery, 'Cost:', vectorian.alignment.LinearGapCost)

class ExponentialGapCostWidget(SlidingGapCostWidget):
    def __init__(self, iquery):
        super().__init__(iquery, 'Cutoff:', vectorian.alignment.ExponentialGapCost, max=20)
                
class GapCostWidget(FineTuneableWidget):
    _description = 'Gap Type:'
    
    _types = [
        ('Constant', ConstantGapCostWidget),
        ('Linear', LinearGapCostWidget),
        ('Exponential', ExponentialGapCostWidget)
    ]
    
    _default = 'Linear'
    
    _box = widgets.HBox
    
class AlignmentAlgorithmWidget:
    def __init__(self, iquery, parameters):
        self._token_metric = TokenSimilarityMetricWidget(iquery)
        
        if parameters is None:
            self._vbox = widgets.VBox([
                self._token_metric.widget])
        else:
            self._vbox = widgets.VBox([
                self._token_metric.widget, parameters])
        
    @property
    def widget(self):
        return self._vbox
    
    def make_token_metric(self):
        return self._token_metric.make()
    
class NeedlemanWunschWidget(AlignmentAlgorithmWidget):
    def __init__(self, iquery):
        self._gap_cost = GapCostWidget(iquery, fix_to="Linear")
        super().__init__(iquery, self._gap_cost.widget)

    def make(self):
        return vectorian.alignment.NeedlemanWunsch(
            gap=self._gap_cost.make().to_scalar())
        
class SmithWatermanWidget(AlignmentAlgorithmWidget):
    def __init__(self, iquery):
        self._gap_cost = GapCostWidget(iquery, fix_to="Linear")
        super().__init__(iquery, self._gap_cost.widget)

    def make(self):
        return vectorian.alignment.SmithWaterman(
            gap=self._gap_cost.make().to_scalar(), zero=self._zero.value)
        
class WatermanSmithBeyerWidget(AlignmentAlgorithmWidget):
    def __init__(self, iquery):
        self._gap_cost = GapCostWidget(iquery)
        self._zero = widgets.BoundedFloatText(
            value=0.25,
            min=0,
            max=1,
            step=0.1,
            description='Zero:',
            disabled=False)
        super().__init__(
            iquery,
            widgets.VBox([self._gap_cost.widget, self._zero]))

    def make(self):
        return vectorian.alignment.WatermanSmithBeyer(
            gap=self._gap_cost.make(), zero=self._zero.value)
        
class WordMoversDistanceWidget(AlignmentAlgorithmWidget):
    _variants = [
        'wmd/kusner',
        'wmd/vectorian',
        'rwmd/kusner',
        'rwmd/jablonsky',
        'rwmd/vectorian'
    ]
    
    # WordMoversDistance.wmd('kusner')

    def __init__(self, iquery):
        self._variant = widgets.Dropdown(
            options=self._variants,
            value="rwmd/vectorian",
            description="Variant:",
            disabled=False)
        
        self._extra_mass_penalty = widgets.FloatText(
            value=-1,
            description='Extra Mass Penalty:',
            disabled=False
        )
        
        super().__init__(widgets.VBox([
            self._variant,
            self._extra_mass_penalty
        ]))

class WordRotatorsDistanceWidget(AlignmentAlgorithmWidget):
    def __init__(self, iquery):
        super().__init__(None)
        
    
class AlignmentWidget(FineTuneableWidget):
    _description = 'Alignment:'

    _types = [
        ('Needleman-Wunsch', NeedlemanWunschWidget),
        ('Smith-Waterman', SmithWatermanWidget),
        ('Waterman-Smith-Beyer', WatermanSmithBeyerWidget),
        ('Word Movers Distance', WordMoversDistanceWidget),
        ('Word Rotators Distance', WordRotatorsDistanceWidget)
    ]

    _default = 'Waterman-Smith-Beyer'
    
    def make_alignment(self):
        return self._fine_tune.make()

    def make_token_metric(self):
        return self._fine_tune.make_token_metric()

    def make(self):
        return vectorian.metrics.AlignmentSentenceMetric(
            token_metric=self.make_token_metric(),
            alignment=self.make_alignment())
    
class TagWeightedAlignmentWidget():
    def __init__(self, iquery):
        self._pos_mismatch_penalty = widgets.FloatSlider(
            value=1,
            min=0,
            max=1,
            step=0.1,
            description='POS Mismatch Penalty:',
            disabled=False)
    
        self._tag_weights = widgets.Dropdown(
            options=['Off', 'POST STSS'],
            value='POST STSS',
            description='Tag Weights:',
            disabled=False)

        self._similarity_threshold = widgets.FloatSlider(
            value=0.2,
            min=0,
            max=1,
            step=0.1,
            description='Similarity Threshold:',
            disabled=False)
        
        self._alignment = AlignmentWidget(iquery)
        
        self._vbox = widgets.VBox([
            self._pos_mismatch_penalty,
            self._tag_weights,
            self._similarity_threshold,
            self._alignment.widget
        ])

    def make(self):
        return vectorian.metrics.TagWeightedAlignmentSentenceMetric(
            token_metric=self._alignment.make_token_metric(),
            alignment=self._alignment.make_alignment())
        
    @property
    def widget(self):
        return self._vbox
        
    
class SentenceEmbeddingWidget:
    # https://docs.google.com/spreadsheets/d/14QplCdTCDwEmTqrn1LH4yrbKvdogK4oQvYO1K1aPR5M/edit#gid=0
    _variants = [
        'stsb-roberta-large',
        'stsb-roberta-base',
        'stsb-bert-large',
        'stsb-distilbert-base'
    ]

    def __init__(self, iquery):
        self._widget = widgets.Dropdown(
            options=self._variants,
            value='stsb-distilbert-base',
            description='Model:',
            disabled=False)

    def make(self):
        return vectorian.metrics.SentenceEmbedding()
        
    @property
    def widget(self):
        return self._widget
    

class SentenceMetricWidget(FineTuneableWidget):
    _description = 'Sentence Metric:'

    _types = [
        ('Alignment', AlignmentWidget),
        ('Tag-Weighted Alignment', TagWeightedAlignmentWidget),
        ('Sentence Embedding', SentenceEmbeddingWidget)
    ]
    
    _default = 'Alignment'

class PartitionWidget:
    def __init__(self, iquery):
        self._iquery = iquery
        
        self._level = widgets.Dropdown(
            options=['sentence', 'token'],
            value='sentence',
            description='Partition:',
            disabled=False)
        
        self._window_size = widgets.BoundedIntText(
            value=1,
            min=1,
            max=1000,
            step=1,
            description='Window Size:',
            disabled=False,
            layout={'width': '10em'})

        self._window_step = widgets.BoundedIntText(
            value=1,
            min=1,
            max=1000,
            step=1,
            description='Window Step:',
            disabled=False,
            layout={'width': '10em'})
        
        self._hbox = widgets.HBox([
            self._level,
            self._window_size,
            self._window_step
        ])
        
    def make(self):
        return vectorian.session.Partition(
            self._iquery.session, self._level.value,
            self._window_size.value, self._window_step.value)
        
    @property
    def widget(self):
        return self._hbox
    
class QueryWidget:
    def __init__(self, iquery):
        self._iquery = iquery
        
        self._query = widgets.Text(
            value='',
            placeholder='Your Query',
            description='Query:',
            disabled=False,
            layout={'width': '40em'})
 
        self._submit_query = widgets.Button(description='Search')
        self._submit_query.on_click(self.on_search)
        
        self._partition = PartitionWidget(iquery)
        self._sentence = SentenceMetricWidget(iquery)

        self._progress = widgets.FloatProgress(
            value=0, min=0, max=1, description='',
            layout=widgets.Layout(width='100%', visibility='hidden'))
        
        self._location_formatter = vectorian.render.location.LocationFormatter()
        
        self._results = widgets.HTML(value='')
        
        self._vbox = widgets.VBox([
            widgets.HBox([self._query, self._submit_query]),
            self._partition.widget,
            self._sentence.widget,
            self._progress,
            self._results])
        
    @property
    def partition(self):
        return self._partition.make()

    @property
    def sentence_metric(self):
        return self._sentence.make()

    @property
    def index(self):
        partition = self._iquery.session.partition(**self.partition.to_args())
        return partition.index(self.sentence_metric, self._iquery.nlp)
    
    def _run_query(self, find, query):
        progress = self._progress

        def update_progress(t):
            progress.value = progress.max * t

        try:
            result = find(
                query,
                progress=update_progress)
        finally:
            progress.close()

        def make_result(*args, **kwargs):
            return vectorian.session.LabResult(
                *args, **kwargs,
                renderers=[vectorian.render.excerpt.ExcerptRenderer()],
                location_formatter=self._location_formatter)

        return make_result, result
            
    def on_search(self, change):
        self._results.value = ''
        self._progress.layout.visibility = 'visible'
        r = self.index.find(
            self._query.value, n=1, run_query=self._run_query)
        self._progress.layout.visibility = 'hidden'
        if r:
            self._results.value = r._repr_html_()
        
    @property
    def widget(self):
        return self._vbox

    
class InteractiveQuery:
    def __init__(self, session, nlp):
        self._session = session
        self._nlp = nlp
        self._widget = QueryWidget(self)
        
    @property
    def session(self):
        return self._session

    @property
    def nlp(self):
        return self._nlp
    
    def set_index(self, index):
        pass
    
    @property
    def widget(self):
        return self._widget.widget
    
    
w = InteractiveQuery(session, nlp)
w.widget